In [21]:
import pandas as pd
import numpy as np
import os

def calculate_response_time_robot_log(file_path, output_path):
    """
    Calculates the response time for the robot_log file.
    Response time is the duration from the start of movement (first pid_output)
    to reaching the target distance.
    """
    print(f"กำลังประมวลผลไฟล์: {file_path}")
    try:
        # Load the CSV file, skipping the first row if it has no data
        # The lambda function checks if the 5th column (pid_output) is empty in the first row
        df = pd.read_csv(file_path, skiprows=lambda x: x > 0 and pd.isna(pd.read_csv(file_path, nrows=1, header=None).iloc[0,4]))
        
        # Drop the initial rows with no PID output to find the true start of movement
        df = df.dropna(subset=['pid_output'])
        
        if df.empty:
            print(f"ไม่พบข้อมูลการเคลื่อนที่ที่ถูกต้องในไฟล์ {file_path}")
            return None

        # The movement starts at the timestamp of the first row with a valid pid_output
        start_time_unix = df['time'].iloc[0]
        
        # Get the target distance from the data
        target_distance = df['target_distance'].iloc[0]
        
        # The movement ends when the relative position is close to the target distance
        # We find the last row where the relative position is within the tolerance (0.02 from the code)
        end_rows = df[abs(df['relative_position'] - target_distance) < 0.02]
        
        if not end_rows.empty:
            # The end time is the timestamp of the last row that meets the condition
            end_time_unix = end_rows['time'].iloc[-1]
            response_time = end_time_unix - start_time_unix
            
            # Print the result
            print(f"Response Time ที่คำนวณได้สำหรับไฟล์ {file_path}: {response_time:.3f} วินาที")
            
            # Create a new column 'response_time_s'
            df['response_time_s'] = 0.0
            
            # Put the calculated response time in the first row of the new column
            # This is done on the original index of the filtered DataFrame
            df.loc[df.index[0], 'response_time_s'] = response_time
            
            # Create the output directory if it doesn't exist
            os.makedirs(output_path, exist_ok=True)
            
            # Save the new file with the new column in the specified output path
            file_name_only = os.path.basename(file_path)
            new_file_path = os.path.join(output_path, file_name_only.replace('.csv', '_with_response_time.csv'))
            df.to_csv(new_file_path, index=False)
            print(f"ข้อมูลที่อัปเดตถูกบันทึกในไฟล์ใหม่: {new_file_path}")
            return response_time
        else:
            print(f"ไม่สามารถคำนวณ Response Time ได้: ไม่พบว่าหุ่นยนต์เคลื่อนที่ถึงระยะเป้าหมายที่ {target_distance} เมตร")
            return None

    except FileNotFoundError:
        print(f"เกิดข้อผิดพลาด: ไม่พบไฟล์ {file_path}")
        return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
        return None

if __name__ == '__main__':
    # Define the output directory
    output_directory = '/Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Data_add_response time'
    
    # List of all files to be processed
    file_list = [
        '/Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_11_03_P2-1_I0-0_D5_ramp0.csv',
        '/Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_10_20_P2-1_I0-0_D1_ramp0.csv',
        '/Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_08_54_P2-1_I5-0_D0_ramp0.csv',
        '/Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_07_43_P2-1_I5-0_D0_ramp0.csv',
        '/Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_07_02_P2-1_I5-0_D0_ramp0.csv',
    ]

    # Process each file in the list
    for file_name in file_list:
        calculate_response_time_robot_log(file_name, output_directory)
        print("-" * 50) # Print a separator for clarity


กำลังประมวลผลไฟล์: /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_11_03_P2-1_I0-0_D5_ramp0.csv
Response Time ที่คำนวณได้สำหรับไฟล์ /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_11_03_P2-1_I0-0_D5_ramp0.csv: 19.063 วินาที
ข้อมูลที่อัปเดตถูกบันทึกในไฟล์ใหม่: /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Data_add_response time/robot_log_23_11_03_P2-1_I0-0_D5_ramp0_with_response_time.csv
--------------------------------------------------
กำลังประมวลผลไฟล์: /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_10_20_P2-1_I0-0_D1_ramp0.csv
Response Time ที่คำนวณได้สำหรับไฟล์ /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_10_20_P2-1_I0-0_D1_ramp0.csv: 19.214 วินาที
ข้อมูลที่อัปเดตถูกบันทึกในไฟล์ใหม่: /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Data_add_response time/robot_log_23_10_20_P2-1_I0-0_D1_ramp0_with_response_time.csv
---------------------------------------

In [22]:
import pandas as pd
import numpy as np
import os

def calculate_response_time_robot_log(file_path, output_path):
    """
    Calculates the response time for the robot_log file.
    Response time is the duration from the start of movement (first pid_output)
    to reaching the target distance.
    """
    print(f"กำลังประมวลผลไฟล์: {file_path}")
    try:
        # Load the CSV file, skipping the first row if it has no data
        # The lambda function checks if the 5th column (pid_output) is empty in the first row
        # โหลดไฟล์ CSV โดยข้ามแถวแรกหากไม่มีข้อมูลในคอลัมน์ที่ 5 (pid_output)
        df = pd.read_csv(file_path, skiprows=lambda x: x > 0 and pd.isna(pd.read_csv(file_path, nrows=1, header=None).iloc[0,4]))
        
        # Drop the initial rows with no PID output to find the true start of movement
        # ลบแถวเริ่มต้นที่ไม่มีค่า PID output เพื่อหาจุดเริ่มต้นของการเคลื่อนที่ที่แท้จริง
        df = df.dropna(subset=['pid_output'])
        
        if df.empty:
            print(f"ไม่พบข้อมูลการเคลื่อนที่ที่ถูกต้องในไฟล์ {file_path}")
            return None

        # The movement starts at the timestamp of the first row with a valid pid_output
        # จุดเริ่มต้นการเคลื่อนที่คือ timestamp ของแถวแรกที่มีค่า pid_output ที่ถูกต้อง
        start_time_unix = df['time'].iloc[0]
        
        # Get the target distance from the data
        # ดึงค่าระยะทางเป้าหมายจากข้อมูล
        target_distance = df['target_distance'].iloc[0]
        
        # The movement ends when the relative position is close to the target distance
        # We find the last row where the relative position is within the tolerance (0.02 from the code)
        # การเคลื่อนที่สิ้นสุดเมื่อตำแหน่งสัมพัทธ์ใกล้เคียงกับระยะทางเป้าหมาย
        # เราหาแถวสุดท้ายที่ตำแหน่งสัมพัทธ์อยู่ในช่วงความคลาดเคลื่อน (0.02)
        end_rows = df[abs(df['relative_position'] - target_distance) < 0.02]
        
        if not end_rows.empty:
            # The end time is the timestamp of the last row that meets the condition
            # เวลาสิ้นสุดคือ timestamp ของแถวสุดท้ายที่ตรงตามเงื่อนไข
            end_time_unix = end_rows['time'].iloc[-1]
            response_time = end_time_unix - start_time_unix
            
            # Print the result
            # พิมพ์ผลลัพธ์
            print(f"Response Time ที่คำนวณได้สำหรับไฟล์ {file_path}: {response_time:.3f} วินาที")
            
            # Create a new column 'response_time_s'
            # สร้างคอลัมน์ใหม่ 'response_time_s'
            df['response_time_s'] = 0.0
            
            # Put the calculated response time in the first row of the new column
            # This is done on the original index of the filtered DataFrame
            # ใส่ค่า response time ที่คำนวณได้ลงในแถวแรกของคอลัมน์ใหม่
            df.loc[df.index[0], 'response_time_s'] = response_time
            
            # Create the output directory if it doesn't exist
            # สร้างไดเร็กทอรีปลายทางหากยังไม่มี
            os.makedirs(output_path, exist_ok=True)
            
            # Save the new file with the new column in the specified output path
            # บันทึกไฟล์ใหม่พร้อมคอลัมน์ใหม่ลงใน path ปลายทางที่กำหนด
            file_name_only = os.path.basename(file_path)
            new_file_path = os.path.join(output_path, file_name_only.replace('.csv', '_with_response_time.csv'))
            df.to_csv(new_file_path, index=False)
            print(f"ข้อมูลที่อัปเดตถูกบันทึกในไฟล์ใหม่: {new_file_path}")
            return response_time
        else:
            print(f"ไม่สามารถคำนวณ Response Time ได้: ไม่พบว่าหุ่นยนต์เคลื่อนที่ถึงระยะเป้าหมายที่ {target_distance} เมตร")
            return None

    except FileNotFoundError:
        print(f"เกิดข้อผิดพลาด: ไม่พบไฟล์ {file_path}")
        return None
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
        return None

if __name__ == '__main__':
    # Define the input and output directories
    # กำหนดไดเร็กทอรีต้นทางและปลายทาง
    input_directory = '/Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data'
    output_directory = '/Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Data_add_response time'
    
    # Get a list of all CSV files in the input directory
    # ดึงรายการไฟล์ CSV ทั้งหมดในไดเร็กทอรีต้นทาง
    file_list = [os.path.join(input_directory, f) for f in os.listdir(input_directory) if f.endswith('.csv')]
    
    # Process each file in the list
    # ประมวลผลไฟล์แต่ละไฟล์ในรายการ
    if not file_list:
        print(f"ไม่พบไฟล์ .csv ในไดเร็กทอรี: {input_directory}")
    else:
        for file_path in file_list:
            calculate_response_time_robot_log(file_path, output_directory)
            print("-" * 50) # Print a separator for clarity / พิมพ์เส้นคั่นเพื่อให้อ่านง่าย


กำลังประมวลผลไฟล์: /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_18_25_17_P2-1_I1_D0_ramp0.csv
Response Time ที่คำนวณได้สำหรับไฟล์ /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_18_25_17_P2-1_I1_D0_ramp0.csv: 19.882 วินาที
ข้อมูลที่อัปเดตถูกบันทึกในไฟล์ใหม่: /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Data_add_response time/robot_log_18_25_17_P2-1_I1_D0_ramp0_with_response_time.csv
--------------------------------------------------
กำลังประมวลผลไฟล์: /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_10_20_P2-1_I0-0_D1_ramp0.csv
Response Time ที่คำนวณได้สำหรับไฟล์ /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Draw_square/data/robot_log_23_10_20_P2-1_I0-0_D1_ramp0.csv: 19.214 วินาที
ข้อมูลที่อัปเดตถูกบันทึกในไฟล์ใหม่: /Users/faiijaran/robot/Robot_Module_2-1/J.Thanet/Data_add_response time/robot_log_23_10_20_P2-1_I0-0_D1_ramp0_with_response_time.csv
---------------------------------------------